# LA Weather App with Weather Predictions
## Carmen Ng, Hailey Sarmiento, Victor Yao

Our project aims to develop a web app that provides real-time, hyperlocal weather visualizations using NASA POWER API data. The app will enable users to explore weather conditions such as temperature, precipitation, wind speed, and humidity for specific locations. Users will be able to select a location and date to retrieve weather trends, compare past weather patterns, and analyze localized climate variations through interactive visualizations.

In addition to real-time and historical weather visualizations, we implemented a Keras-based neural network model to predict future weather conditions. The model is trained using historical weather data obtained from the NASA POWER API. It learns patterns in temperature, humidity, wind speed, and precipitation over time, allowing it to make short-term forecasts for a given location. **(explain more, idk)**

In order to implement this app we utilized a combination of Python tools and data visualization techniques. API requests (aiohttp, asyncio), plotly, keras/tensorflow, Dash. **(possibly Diskcache)**

In [1]:
# make sure to run these first 
# pip install aiohttp

In [30]:
# import necessary libraries first
import numpy as np
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import express as px
import plotly.graph_objs as go
from dash import Dash, html, Input, Output, Patch, dash_table
import dash_leaflet as dl
import json
from dash import Dash, dcc, html, dash_table, Input, Output, State, callback, no_update
import base64
import datetime
from datetime import date, datetime, timedelta
import io
from flask import Flask, jsonify
import asyncio
from threading import Thread
import aiohttp
import nest_asyncio
from dash.dash_table import DataTable, FormatTemplate
from dash.dash_table.Format import Format, Scheme, Trim

# NASA API

To get the weather data needed for this app, we use the temporal API from NASA Power (Prediction of Worldwide Energy Sources). This provides accurate weather and climate data from NASA's satellites, and can be requested daily or hourly.

## How does the NASA API call work?

NASA Power API Documentation can be found here: https://power.larc.nasa.gov/docs/services/api/

How this API works is that you send a request to NASA's servers and they return the paramaters you ask for in a JSON format that we can use for our data visualizations and model later on. In the request, you need to specify your location (latitude and longitude), the time period (choosing between daily, hourly, or long-term averages), the weather parameters you want, and the format (here we chose JSON). 

We first identified what parameters we wanted to use throughout our project by looking at the parameter dictionary. 

Parameter Dictionary can be found here: https://power.larc.nasa.gov/docs/tutorials/parameters/

We turned this into an API call that we can use to call for any location and time in our app. When the user inputs their location on the map, the map sends the coordinates to our API function, which then finds the data for the paramaters we requested. This information is then used in our visualizations.

In [26]:
# NASA API

def get_nasa_power_data(lat, lon, start_date, end_date, for_graph=False):
    """
    Fetches NASA POWER API data for given latitude, longitude, and time range.

    Args:
    - lat (float): Latitude of the location.
    - lon (float): Longitude of the location.
    - start_date (str): Start date in YYYYMMDD format.
    - end_date (str): End date in YYYYMMDD format.

    Returns:
    - Pandas DataFrame with selected weather parameters.
    """

    # Specify multiple parameters in the API request
    parameters = "PRECSNO,T2MDEW,PRECTOTCORR,T2M,WS2M,RH2M,CLOUD_AMT"

    url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "parameters": parameters,
        "community": "RE",
        "longitude": lon,
        "latitude": lat,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }

    response = requests.get(url, params=params)

    data = response.json()

    # Convert JSON response to DataFrame and transpose it
    nasa_weather = pd.DataFrame.from_dict(data["properties"]["parameter"], orient="index").T

    # Reset index and rename date column
    nasa_weather.reset_index(inplace=True)
    nasa_weather.rename(columns={"index": "date"}, inplace=True)

    # Convert date column to proper datetime format
    nasa_weather["date"] = pd.to_datetime(nasa_weather["date"], format="%Y%m%d", errors="coerce")
    nasa_weather.dropna(subset=["date"], inplace=True)  # Remove invalid date rows

    nasa_weather.rename(columns={
        "PRECSNO": "Snow_Precipitation",
        "T2MDEW": "Dew_Point_2m",
        "PRECTOTCORR": "Total_Precipitation_mm",
        "T2M": "Temperature_2m_C",
        "WS2M": "Wind_Speed_2m",
        "RH2M": "Relative Humidity (%)",
        "CLOUD_AMT" : "Cloud Cover (%)",
    }, inplace=True)
    
    # Add Rounded_Lat and Rounded_Lng for merging
    nasa_weather['Rounded_Lat'] = lat
    nasa_weather['Rounded_Lng'] = lon
    
    nasa_weather['Precipitation (in)'] = nasa_weather['Total_Precipitation_mm'] / 25.4 # mm to in
    nasa_weather['Temperature (F)'] = (nasa_weather['Temperature_2m_C'] * (9./5.)) + 32. # C to F
    nasa_weather['Wind Speed (mph)'] = nasa_weather['Wind_Speed_2m'] * 2.237 # m/s to mph

    if for_graph:
        nasa_weather = nasa_weather[["Precipitation (in)","Temperature (F)","Wind Speed (mph)","Relative Humidity (%)","Cloud Cover (%)"]]
        # nasa_weather["Temperature (F)"] = round(nasa_weather["Temperature (F)"],2)
        # nasa_weather["Precipitation (in)"] = round(nasa_weather["Precipitation (in)"],2)

    return nasa_weather

# Complex Visualizations Functions

We used the Plotly library to visualize weather data retrieved from the NASA POWER API for the Dash app.  

Because retrieving all this data sequentially would have been too slow, I used asynchronous requests instead of the synchronous one we created before to significantly improve speed and performance.

## How does aiohttp and asyncio work?

If you want to learn more about aiohttp and asyncio:
- aiohttp Documentation: https://docs.aiohttp.org/en/stable/
- asyncio Documentation: https://docs.python.org/3/library/asyncio.html

When making API requests, you usually use 'requests.get', which waits for each request to finish before requesting the next one. We decided to use aiohttp and asyncio to fetch multiple requests simultaneously because requesting weather data from each coordinate one by one took too long. This made making each visualization much faster from the requested data.

### What is asyncio?
asyncio is a Python library for running asynchronous tasks. It allows multiple tasks to run at the same time. It can work for API requests (what we are using it for) or for downloading files. 

Basic rundown of how to use:
1. `async def` -> defines the function (task) you want to do
2. `await` -> tells Python to wait for a task before continuing
3. `asyncio.gather()`-> runs multiple tasks at the same time

### What is aiohttp?
aiohttp works with asyncio to send multiple requests at the same time, this makes fetching API data faster because they all run at the same time. It replaces 'requests.get() for async operations. 

Basic rundown of how to use:
1. `aiohttp.ClientSession()` -> open a session to send requests
2. `async with session.get(url) as response` -> Fetches API data asynchronously
3. `await response.json()` -> waits for API response and converts it to JSON (automatically converts the JSON response into a Python dictionary)

In [20]:
# Apply nest_asyncio for async execution 
nest_asyncio.apply()

async def fetch_nasa_data(session, lat, lon, start_date, end_date, sem):
    """
    Fetches hourly weather data from the NASA POWER API asynchronously for a given location and time.

    Args:
        session (aiohttp.ClientSession): The active aiohttp session for making HTTP requests.
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.
        start_date (str): The start date in YYYYMMDD format.
        end_date (str): The end date in YYYYMMDD format.
        sem (asyncio.Semaphore): A semaphore to limit the number of concurrent API requests.

    Returns:
        dict: A dictionary containing the API response data, or None if an error occurs.
    """
    # request information from nasa power
    url = "https://power.larc.nasa.gov/api/temporal/hourly/point"
    
    # define the parameters that we want to call
    parameters = "PRECSNO,T2MDEW,PRECTOTCORR,T2M,WS2M,RH2M,CLOUD_AMT"
    
    # Construct the API request parameters
    params = {
        "parameters": parameters,
        "community": "RE",
        "longitude": lon,
        "latitude": lat,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }
    
    # Use a semaphore to control the number of simultaneous API requests
    async with sem:  
        # Send an asynchronous request to the NASA POWER API
        async with session.get(url, params=params) as response:
            # print error if no data is found for location
            if response.status != 200:
                print(f"Error {response.status} for {lat}, {lon}, {start_date}-{end_date}")
                return None
            # Convert the API response to a dictionary and return it
            return await response.json()  

async def get_nasa_power_hourly_data(lat, lon, date=None):
    """
    Fetches NASA POWER hourly weather data for a single day.

    Args:
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.
        date (str, optional): Date in YYYYMMDD format. (Defaults to today's date)

    Returns:
        pd.DataFrame: A DataFrame containing the weather data.
    """
     # Use today's date if none is provided
    if date is None:
        date = datetime.now().strftime("%Y%m%d")
        
    # Create a semaphore to limit simultaneous API requests (prevents timeouts)
    sem = asyncio.Semaphore(5)
    
    # Open an asynchronous session to send the API request
    async with aiohttp.ClientSession() as session:
        # Fetch weather data from NASA POWER API asynchronously
        # 'await' ensures this task completes before moving to the next step
        data = await fetch_nasa_data(session, lat, lon, date, date, sem)
    
    # Check if the response is empty or missing the 'properties' key
    if data is None or "properties" not in data:
        print(f"No data found for {lat}, {lon} on {date}")
        return None
    
    # Convert the JSON response to a Pandas DataFrame
    df = pd.DataFrame.from_dict(data["properties"]["parameter"], orient="index").T
    df.reset_index(inplace=True)
    df.rename(columns={"index": "datetime"}, inplace=True)
    
    # Convert datetime strings to actual datetime objects & remove invalid value
    df["datetime"] = pd.to_datetime(df["datetime"], format="%Y%m%d%H", errors="coerce")
    df.dropna(subset=["datetime"], inplace=True)
    
    # Rename columns
    df.rename(columns={
        "PRECSNO": "Snow_Precipitation",
        "T2MDEW": "Dew_Point_2m",
        "PRECTOTCORR": "Total_Precipitation_mm",
        "T2M": "Temperature_2m_C",
        "WS2M": "Wind_Speed_2m",
        "RH2M": "Relative Humidity (%)",
        "CLOUD_AMT" : "Cloud Cover (%)",
    }, inplace=True)

    # Add Lat/Lon
    df["Latitude"] = lat
    df["Longitude"] = lon

    # Convert units
    df["Precipitation(in)"] = df["Total_Precipitation_mm"] / 25.4  
    df["Temperature(F)"] = (df["Temperature_2m_C"] * 9/5) + 32  
    df["Wind_Speed(mph)"] = df["Wind_Speed_2m"] * 2.237  
    
    # Return the cleaned DataFrame for use in visualizations
    return df

async def get_nasa_power_hourly_data_years(lat, lon, start_year, end_year):
    """
    Fetches NASA POWER API hourly data for multiple years in 2-year chunks to prevent timeouts.

    Args:
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.
        start_year (int): The starting year of data collection.
        end_year (int): The ending year of data collection.

    Returns:
        pd.DataFrame: A DataFrame containing the weather data for multiple years
    """
    # Create a list to store asynchronous API requests
    tasks = []
    
    # loop through specified range of years
    for year in range(start_year, end_year): 
        # start January 1st of the first year
        start_date = f"{year}0101"
        # end at December 31st of the last year
        end_date = f"{min(year, end_year)}1231"
        # Add an asynchronous task to fetch weather data for this year
        tasks.append(get_nasa_power_hourly_data(lat, lon, start_date))
    
    # Execute all API requests concurrently
    results = await asyncio.gather(*tasks)
    
    # Filter out None values and combine results
    all_data = [df for df in results if df is not None]

    # Return a combined DataFrame with weather data for multiple years
    # If no valid data is found, return None
    return pd.concat(all_data, ignore_index=True) if all_data else None


In [5]:
def generate_nearby_locations(lat, lon, radius_miles=15, num_points=10):
    """
    Generates random nearby locations within a radius using NumPy.

    Args:
    - lat (float): Latitude of the center point.
    - lon (float): Longitude of the center point.
    - radius_miles (float): Search radius in miles.
    - num_points (int): Number of locations to generate.

    Returns:
    - NumPy array of shape (num_points, 2) with latitude and longitude values.
    """
    # Generate random angles uniformly between 0 and 2π
    angles = np.random.uniform(0, 2 * np.pi, num_points)

    # Use square root scaling to distribute points uniformly across the circle
    distances = radius_miles * np.sqrt(np.random.uniform(0, 1, num_points))

    # Convert polar coordinates (distance, angle) to latitude & longitude offsets
    delta_lat = (distances / 69) * np.cos(angles)
    delta_lon = (distances / (69 * np.cos(np.radians(lat)))) * np.sin(angles)

    # Compute new latitudes and longitudes
    new_lat = lat + delta_lat
    new_lon = lon + delta_lon

    return np.column_stack((new_lat, new_lon))

In [6]:
async def get_nasa_power_data_nearby(lat, lon, date=None, num_locations=10, radius_miles=15):
    """
    Fetches NASA POWER API hourly data for multiple nearby locations for a single day.

    Args:
        lat (float): Latitude of the central location.
        lon (float): Longitude of the central location.
        date (str, optional): Date in "YYYYMMDD" format. Defaults to today.
        num_locations (int, optional): Number of nearby locations to generate. Default is 10.
        radius_miles (float, optional): Radius in miles to generate nearby locations. Default is 15.

    Returns:
        pd.DataFrame or None: A Pandas DataFrame containing weather data for all locations
    """
    # If no date is provided, use today's date in YYYYMMDD format
    if date is None:
        date = datetime.now().strftime("%Y%m%d")
        
    # Generate a list of random nearby locations within the specified radius
    locations = generate_nearby_locations(lat, lon, radius_miles, num_locations)
    
    # Create a list of asynchronous tasks to fetch weather data for each location
    tasks = [get_nasa_power_hourly_data(location[0], location[1], date) for location in locations]
    
    # Fetch data for all locations concurrently
    results = await asyncio.gather(*tasks)
    
    # Remove any None values
    all_data = [df for df in results if df is not None]
    
    # Combine all the retrieved data into a single Pandas DataFrame
    # If no valid data was found, return None
    return pd.concat(all_data, ignore_index=True) if all_data else None


For the visualizations, I created plots showing hourly updates for temperature, precipitation, and wind speed for the user-selected location. I used the `get_nasa_power_hourly_data(lat, lon, specific_date)` function to retrieve the necessary data. This function allowed me to obtain the parameters for a specific day at the chosen location.  

I also created a visualization displaying weather trends over the past ten years. To make the data more readable, I calculated the mean values for precipitation, temperature, and wind speed. To implement this, I created a new function call: `get_nasa_power_hourly_data_years(lat, lon, start_year, end_year)`. This function retrieves two years of data at a time because requesting too many years from the NASA POWER API at once caused timeouts.  

Lastly, I created a visualization using Plotly’s choropleth map to display precipitation and temperature at random locations within a radius of the user’s selected location. I chose this approach because generating a heatmap required making API requests for every coordinate, which was not feasible. To implement this, I created a new function. The `get_nasa_power_data_nearby()` function fetches weather parameters for nearby locations after the `generate_new_locations()` function generates a set of coordinates within a 15-mile radius of the user’s chosen location.

In [7]:
# Dictionary mapping weather attributes to their corresponding line colors for visualization
attr_colors = {"Temperature (F)": "red", 
         "Precipitation (in)":"blue", 
         "Wind Speed (mph)":"green", 
         "Relative Humidity (%)":"teal", 
         "Cloud Cover (%)":"grey"}

In [8]:
def plot_hourly(df, attr):
    """
    Generates a line plot of an hourly weather attribute over time.

    Args:
        df (pd.DataFrame): The DataFrame containing the weather data.
        attr (str): The weather attribute to plot 

    Returns:
        fig: A figure displaying the hourly trends for the selected attribute.
    """
    fig = px.line(df, x="datetime", y=attr, 
                  title=f"Hourly {attr} Trends",
                  labels={"datetime": "Datetime"},
                  line_shape='linear', 
                  # Set line color based on attribute type
                  color_discrete_sequence=[attr_colors[attr]])
    
    # Update the layout
    fig.update_layout(xaxis_title="Datetime", 
                      yaxis_title=attr, 
                      xaxis_tickangle=-45)
    
    # Return the figure
    return fig

In [9]:
def plot_yearly(df, attr):
    """
    Generates a box plot to visualize yearly trends of a weather attribute.

    Args:
        df (pd.DataFrame): The DataFrame containing weather data.
        attr (str): The weather attribute to plot (must be a key in attr_colors).

    Returns:
        fig: A figure displaying the yearly trends of the selected attribute.
    """
    # Create a new 'year' column
    df["year"] = df["datetime"].dt.year
    
    # Create a box plot
    fig = px.box(df,
                 x="year",
                 y=attr,
                 # Set color based on attribute type
                 color_discrete_sequence=[attr_colors[attr]],
                 # update labels
                 title=f"Yearly Average {attr} Trends",
                 labels={"year": "Year"})
    
    # return the figure
    return fig


In [10]:
def plot_mock_heatmap(attr, lat, lon, date=None, num_locations=50, radius_miles=15):
    """
    Fetches weather data for locations in a specified radius and plots a scatter map 
    showing temperature, precipitation, or other weather attributes.

    Args:
        attr (str): The weather attribute to visualize.
        lat (float): Latitude of the central location.
        lon (float): Longitude of the central location.
        date (str, optional): Date in "YYYYMMDD" format. Defaults to today.
        num_locations (int, optional): Number of nearby locations to generate.
        radius_miles (float, optional): Radius in miles for generating nearby locations.

    Returns:
        fig: A scatter map visualization of the selected weather attribute.
    """
    # Fetch weather data for the generated locations using asyncio
    # use get_nasa_power_data_nearby() to get data for attribute for nearby locations
    weather_data = asyncio.run(get_nasa_power_data_nearby(lat, lon, date, num_locations, radius_miles))
    # print error if data is not found
    if weather_data is None or weather_data.empty:
        print("No data retrieved.")
        return
  
    # Create scatter map visualization
    fig = px.scatter_mapbox(
        weather_data, lat="Latitude", lon="Longitude", color=attr,
        title=f"{attr} in Nearby Locations"
    )
    
    # set style of map
    fig.update_layout(
        mapbox_style="carto-positron",
        autosize=True 
    )
    
    # return map visualization
    return fig


# Keras Model to Predict Upcoming Weather

In [11]:
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
#from keras import layers, losses
from keras.models import Sequential
from keras.layers import LSTM, Dense,Dropout, Bidirectional
from sklearn.preprocessing import MinMaxScaler
import datetime
import requests
"""
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.layers.recurrent import LSTM
"""
import matplotlib.pyplot as plt
from plotly import express as px
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, mean_absolute_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import preprocessing, tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [12]:
# Function that takes a feature, and runs a model to predict the next 4 days of that feature based on 30 days of previous weather
def run_model(dataset, feature, n_past=50, n_future=7):
    feature = feature
    dataset = dataset.dropna(subset=[feature])
    dataset = dataset.reset_index(drop=True)
    training_set = dataset[[feature]].values

    sc = MinMaxScaler(feature_range=(0,1))
    training_set_scaled = sc.fit_transform(training_set)

    x_train = []
    y_train = []

    n_past = n_past # Number of days to use as training data
    n_future = n_future # Number of days into future to predict feature

    for i in range(0,len(training_set_scaled)-n_past-n_future+1):
        x_train.append(training_set_scaled[i : i + n_past , 0])
        y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
    x_train , y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

    model = Sequential([
        Bidirectional(LSTM(units=n_past, return_sequences=True, input_shape=(x_train.shape[1], 1))),
        Dropout(0.2),
        LSTM(units=n_past, return_sequences=True),
        Dropout(0.2),
        LSTM(units=n_past, return_sequences=True),
        Dropout(0.2),
        LSTM(units=n_past),
        Dropout(0.2),
        Dense(units=n_future, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
    model.fit(x_train, y_train, epochs=20, batch_size=32)

    testdataset = dataset.copy()
    testdataset = testdataset[[feature]].iloc[:n_past].values
    # return n_past most recent temperatures 
    real_temperature = dataset.copy().sort_values("date", ascending=False).iloc[-n_past:-1]
    real_temperature = real_temperature[["date",feature]]
    # real_temperature = real_temperature[["date",feature]].sort_values("date", ascending=False).iloc[-n_past:-1].values
    testing = sc.transform(testdataset)
    testing = np.array(testing)
    testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))

    predicted_temperature = model.predict(testing)
    predicted_temperature = sc.inverse_transform(predicted_temperature)
    predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))
    return real_temperature, predicted_temperature

# real_temperature, predicted_temperature = run_model(dataset2, "Temperature(F)", 300, 30)

In [13]:
from datetime import date, datetime, timedelta
import plotly.express as px

def plot_predictions(attr, real, predicted, n_future):
    """
    Plots a line graph of past weather attributes over the given date range selected by
    the user and the n_future days of predicted weather attributes
    
    Args:
        attr (str): the weather attribute of interest to plot and predict
        real (dataframe): a dataframe with the weather attribute on a specific date
        predicted (np.array): an array of predicted weather attributes of length n_future
        n_future (int): the number of days into the future with predicted weather
        
    Returns:
        fig (plotly line graph): real and predicted weather attributes plotted over time
        df (dataframe): a dataframe with labeled real and predicted weather points for 
            specific dates
    """
    # add real labels to weather attr of dates that came directly from nasa api database
    real["Label"] = "Real"
    
    # create list of n_future number of new dates the model is predicting
    new_dates = [real["date"].iloc[0] + timedelta(days=x) for x in range(1,n_future+1)]
    
    # create a temporary dataframe from a dict of dates, predicted weather, and labels
    new_rows = pd.DataFrame({"date": new_dates,
                             attr: predicted.flatten(), 
                             "Label": "Predicted"})
    
    # concatenate real and predicted dataframes with all columns filled
    df = pd.concat([real, new_rows], ignore_index=True)
    
    # plot a line graph where real and predicted weather are differentiated by color
    fig = px.line(df,x="date",y=attr,color="Label",markers=True,symbol="Label",
                  hover_data = {attr:':.4f'},) # round hover data
    
    # create dataframe subset with only predicted dates to display in table
    df = df[df["Label"]=="Predicted"]
    df[attr] = round(df[attr],2)
    
    # make dates into familiar month-day-year format for display in table
    df["Date"] = df["date"].dt.strftime("%m-%d-%Y")
    
    # remove labels from records to display in table
    df = df[["Date",attr]]
    
    return fig, df

In [14]:
# dataset2 = get_nasa_power_data(34.05, -118.25, "20220620", "20230621")
# real_temperature, predicted_temperature = run_model(dataset2, "Temperature (F)", 50, 30)
# graph = plot_predictions("Temperature (F)", real_temperature, predicted_temperature, 30)
# graph.show()

In [15]:
# make predictions graph have the actual data if it is avaiable :)))
# would be a very cool grpah to verify as well

## Interactive Weather App (in progress)

I found that  Dash does not support async callbacks by default, but you can use diskcache to make async Dash callbacks without blocking the main app. I put maybe potential code to do this below but I have not tested it that well yet

In [16]:
'''
import dash
import diskcache
from dash.long_callback import DiskcacheLongCallbackManager

# Create a cache to store async callback results
cache = diskcache.Cache("./cache")
long_callback_manager = DiskcacheLongCallbackManager(cache)

app = dash.Dash(__name__, long_callback_manager=long_callback_manager)

@app.callback(
    Output("visual", "figure"),
    Output("caption", "children"),
    Input("submit-graph", "n_clicks"),
    State("options", "value"),
    State("graph_type", "value"),
    State("latitude", "data"),
    State("longitude", "data"),
    State("date", "data"),
    background=True,  # Runs callback asynchronously
    prevent_initial_call=True,
)
async def create_graph(n_clicks, attr, graph_type, lat, lng, day):
    """
    Asynchronous callback function to create graphs without blocking Dash UI.
    """
    fig = await choose_graph(attr, graph_type, lat, lng, day)  # Async function call
    return fig, f"Displaying {attr} as {graph_type}"
'''


'\nimport dash\nimport diskcache\nfrom dash.long_callback import DiskcacheLongCallbackManager\n\n# Create a cache to store async callback results\ncache = diskcache.Cache("./cache")\nlong_callback_manager = DiskcacheLongCallbackManager(cache)\n\napp = dash.Dash(__name__, long_callback_manager=long_callback_manager)\n\n@app.callback(\n    Output("visual", "figure"),\n    Output("caption", "children"),\n    Input("submit-graph", "n_clicks"),\n    State("options", "value"),\n    State("graph_type", "value"),\n    State("latitude", "data"),\n    State("longitude", "data"),\n    State("date", "data"),\n    background=True,  # Runs callback asynchronously\n    prevent_initial_call=True,\n)\nasync def create_graph(n_clicks, attr, graph_type, lat, lng, day):\n    """\n    Asynchronous callback function to create graphs without blocking Dash UI.\n    """\n    fig = await choose_graph(attr, graph_type, lat, lng, day)  # Async function call\n    return fig, f"Displaying {attr} as {graph_type}"\n'

In [17]:
visual_options = ["Temperature (F)", "Precipitation (in)", "Wind Speed (mph)", "Relative Humidity (%)", "Cloud Cover (%)"]
graph_options = ["Hourly Change", "Yearly Change", "Local Trends"]

async def choose_graph(attr, graph_type, lat, lng, day):
    """
    Choose which graph to plot and send as output to dcc.Graph in dash app
    based off of the weather attribute, graph type, location, and day chosen
    by the user. Fetches weather dataframes using async functions
    
    Args:
        attr (string): the weather attribute of interest to plot and predict
        graph_type (string): the type of graph (hourly, yearly, or heatmap) to plot
        lat (double): latitude coordinate representing location
        lng (double): longitude coordinate representing location
        day (str): string date in %Y%m%d format to fetch nasa api data
        
    Returns:
        A specified graph plotted using user-defined parameters
    
    """    
    # to plot hourly change line graphs in weather attr
    if graph_type == "Hourly Change":
        hour_df = await get_nasa_power_hourly_data(lat, lng, day)
        return plot_hourly(hour_df,attr)
    
    # to plot yearly average change boxplots in weather attr
    elif graph_type == "Yearly Change":
        year_df = await get_nasa_power_hourly_data_years(lat, lng, 2014, 2024)
        return plot_yearly(year_df,attr)

    # to plot heatmaps of weather attr in nearby locations 
    else: 
        return plot_mock_heatmap(attr, lat, lng, day, num_locations=50)

In [34]:
app = Dash()
app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Current Weather', children=[
            dcc.Markdown('''
                ## Directions: 
                1. Pan and zoom in/out on the map to find your location.
                2. Click your location on the map for current weather.
                    Press the "Submit" button to confirm your selection and your weather reuslts will load promptly.
                3. Check out the "Weather Graphs" and "Weather Predictions" tabs for more weather information.'''),
            dcc.Markdown('''## Select Location:'''),
            dl.Map( # add zoom to city buttons?
                id='map',
                n_clicks=0,
                children=[
                    dl.TileLayer()
                ],
                center=[34, -118],
                zoom=9,
                style={'height': '50vh'}
            ),
            html.Div(id='coords'),
            dcc.Markdown("## Select Date:"),
            dcc.DatePickerSingle(
                id='date-picker',
                min_date_allowed=date(2000, 1, 1),
                max_date_allowed=date(2024, 12, 31),
                initial_visible_month=date(2024, 1, 1),
            ),
            html.Button('Submit',id='submit',n_clicks=0), 
            dcc.Markdown('## Current Weather:'),
            dcc.Markdown(id='table-caption'),
            dash_table.DataTable(id='weather',
                style_as_list_view=True,
                style_cell={'padding': '2px'},
                style_header={
                    'backgroundColor': 'white',
                    'fontWeight': 'bold'
                },
                style_cell_conditional=[
                    {'textAlign': 'center'}
                ],
            ),
            dcc.Store(id='latitude'),
            dcc.Store(id='longitude'),
            dcc.Store(id='date'),
        ]),
        
        dcc.Tab(label='Weather Graphs', children=[
            dcc.Markdown('''
                ## Directions: 
                1. Select a weather attribute you are interested in.
                2. Select the type of graph you wish to see.
                3. Click the "Create Graph" button to confirm your choice. 
                    Your graph will load promptly with weather information from your previously selected location.'''),
            dcc.Markdown("### Select an Attribute:"),
            dcc.Dropdown(visual_options,id='options'),
            dcc.Markdown("### Select a Graph:"),
            dcc.Dropdown(graph_options,id='graph_type'),
            html.Button('Create Graph', id='submit-graph', n_clicks=0),
            dcc.Markdown(id='caption'),
            dcc.Graph(id='visual'),
        ]),
        
        dcc.Tab(label='Weather Predictions', children=[
            dcc.Markdown('''
                ## Directions: 
                1. Select a weather attribute you are interested in.
                2. Select the number of days into the future you want to be predicted.
                3. Click the "Predict Weather" button to confirm your choice. 
                    Your graph will load promptly with weather information from your previously selected location.'''),
            dcc.Markdown("### Select an Attribute:"),
            dcc.Dropdown(visual_options,id='target'),
            dcc.Markdown("### Select the Number of Days to Predict:"),
            dcc.Input("Number of Predicted Days", id='n_future', type="number"),
            html.Button('Predict Weather', id='predict-weather', n_clicks=0),
            dcc.Markdown(id='pred-caption'),
            dash_table.DataTable(id='pred-table',
                style_as_list_view=True,
                style_cell={'padding': '2px'},
                style_header={
                    'backgroundColor': 'white',
                    'fontWeight': 'bold'
                },
                style_cell_conditional=[
                    {'textAlign': 'left'}
                ],
            ),
            dcc.Graph(id='pred-graph'),
        ])
    ])
])
    
# TAB 1 - CURRENT WEATHER ---------------------------------------------------------------------------------
@app.callback(
    Output('coords', 'children'), # output for coordinate str
    Output('map', 'children'), # output for markers
    Output('latitude','data'), # store latitude for other tabs/function
    Output('longitude','data'), # store longitude for other tabs/functions
    Input('map', 'clickData'), # update with new coords
    Input('map', 'n_clicks'), # update with new clicks
    prevent_initial_call=True
)
def map(click_data, n_clicks):
    if n_clicks > 0: # runs only if 'submit' button is pressed
        
        # save coordinates from map click data
        coordinates = click_data['latlng']
        latitude, longitude = coordinates.values()
        
        # create Patch() instance, add Marker layer at click coordinates
        patched = Patch()
        patched.append(dl.Marker(position=[latitude, longitude]))
        
        return json.dumps(coordinates), patched, latitude, longitude
    
@app.callback(
    Output('weather','data'), # records to display in table
    Output('weather','columns'), # columns to display in table
    Output('date','data'), # store date str for other tabs
    Output('table-caption','children'),
    Input('submit','n_clicks'), # run when button is pressed
    State('latitude','data'), # latitude from 'current-weather' tab
    State('longitude','data'), # longitude from 'current-weather' tab
    State('date-picker','date'), # user-selected date
    prevent_initial_call=True,
)
def submit(n_clicks, lat, lng, day):
    if n_clicks > 0: # runs only if 'submit' button is pressed
        
        # displays warning if location is not chosen before
        if(lat == None or lng == None):
            return '##### Please select a location.'
        
        # date to formated date for api call
        chosen_date = date.fromisoformat(day).strftime('%Y%m%d')
        
        # return single row df with current weather
        df = get_nasa_power_data(lat, lng, chosen_date, chosen_date, for_graph=True)
        
        # extract column names and records for table, round numbers to 4 significant figures
        cols = [{'name': col, 'id': col, 'type':'numeric', 'format': Format(precision=4)} for col in df.columns]
        weather_df = df.to_dict('records')
        
        caption = f"for **({round(lat,2)}, {round(lng,2)})** on **{date.fromisoformat(day).strftime('%B %d, %Y')}**"
        
        return weather_df, cols, chosen_date, caption
        
# TAB 2 - WEATHER GRAPHS ---------------------------------------------------------------------------------
@app.callback(
    Output('visual','figure'), # graph of weather attribute
    Output('caption','children'), # caption with parameters
    Input('submit-graph','n_clicks'), # run when button is pressed
    State('options','value'), # weather attribute to plot
    State('graph_type','value'), # graph type to plot
    State('latitude','data'), # latitude from 'current-weather' tab
    State('longitude','data'), # longitude from 'current-weather' tab
    State('date','data'), # date from 'current-weather' tab
    prevent_initial_call=True,
)
def create_graph(n_clicks, attr, graph_type, lat, lng, day):
    if n_clicks > 0: # runs only if 'submit-graph' button is pressed
        
        # convert date from num string to readable date
        chosen_date = date.fromisoformat(day).strftime('%B %d, %Y')
        
        # caption to inform user of what parameters they selected
        caption = f" **{attr}** graph for **{chosen_date}** at **({round(lat,2)}, {round(lng,2)})**."
        
        # for async call of fetch_nasa_data
        loop = asyncio.new_event_loop()
        graph = loop.run_until_complete(choose_graph(attr, graph_type, lat, lng, day))
        
        return graph, caption

    
# TAB 3 - WEATHER PREDICTIONS ---------------------------------------------------------------------------------
@app.callback(
    Output('pred-caption','children'),      # caption with parameters
    Output('pred-graph','figure'),          # graph of real and predicted weather
    Output('pred-table','data'),            # weather_df with real and predicted values
    Output('pred-table','columns'),         # cols of weather_df for table
    Input('predict-weather','n_clicks'),    # run when button is pressed
    State('target','value'),                # weather attribute to predict and plot
    State('n_future','value'),              # number of days into the future to predict
    State('latitude','data'),               # latitude from 'current-weather' tab
    State('longitude','data'),              # longitude from 'current-weather' tab
    State('date-picker','date'),            # date from 'current-weather' tab
    prevent_initial_call=True,
)
def get_predictions(n_clicks, attr, n_future, lat, lng, day):
    if n_clicks > 0: # runs only if 'predict-weather' button is pressed
        
        # calculate dates a year apart for get_nasa_power_data
        start = (date.fromisoformat(day) - timedelta(days=365)).strftime('%Y%m%d')
        end = date.fromisoformat(day).strftime('%Y%m%d')
        
        # caption to inform user of what parameters they selected
        caption = f" **{attr}** predictions at **({round(lat,2)}, {round(lng,2)})** using data from **{start}** to **{end}**."
        
        # get dataframe with year of nasa weather data
        dataset = get_nasa_power_data(lat, lng, start, end)
        
        # get real dataframe with known weather, get predicted data array from model
        real, predicted, = run_model(dataset, attr, 50, int(n_future))
        
        # return graph and dataframe with predicted and real data
        graph, pred_df = plot_predictions(attr, real, predicted, n_future)
        
        # extract column names and records to display as table
        cols = [{'name': col, 'id': col} for col in pred_df.columns]
        weather_df = pred_df.to_dict('records')
        
        return caption, graph, weather_df, cols

if __name__ == '__main__':
    app.run(port=1321,debug=True)